<a href="https://colab.research.google.com/github/XplorodoX/EDS_projekt/blob/master/BMI_Uebergewicht.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Einführung in Data Science**

**Fakultät Informatik &ndash; Hochschule Aalen**

**Autoren:** Florian Merlau, Vithusan Ratneswaran, Gianni Balbo

# **Explorative Datenanalyse:**

## **Erste Schritte**

### *Entscheidung welche features wir brauchen*


Wir haben uns für die features ERBMI, ... entschieden, weil wir die Essstörungen im Bezug zum BMI analysieren wollen.

## ***Vorbereitung fürs Coden***



### *Benötigte Bilbliotheken importieren*

In [396]:
!pip install iteration-utilities

In [397]:
import pandas as pd
import numpy as np
from iteration_utilities import duplicates
from iteration_utilities import unique_everseen
import plotly.express as px
import plotly.graph_objects as go
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from google.colab import files
from zipfile import ZipFile
import requests

### Collect/Upload Datenbanken (nur wenn die Laufzeit neu initalisiert wird!)

In [398]:
def get_Databank():
  uploaded = files.upload()

#get_Databank()

### *Datenbanken importieren*







In [399]:
atusrost_link = 'https://www.bls.gov/tus/special.requests/atusrost_2016.zip'
atussum_link = 'https://www.bls.gov/tus/special.requests/atussum_2016.zip'

In [445]:
raw_ehresp = pd.read_csv('ehresp_2016s.dat')
raw_atusrost = pd.read_csv('atusrost_2016.dat')
raw_atussum = pd.read_csv('atussum_2016.dat')

## **Analyse der Datenbanken**

### *Auslesen der Tabellen mit Head() zum Inspizieren*

In [401]:
raw_ehresp.head()

,TUCASEID,TULINENO,EEINCOME1,ERBMI,ERHHCH,ERINCOME,ERSPEMCH,ERTPREAT,ERTSEAT,ETHGT,ETWGT,EUDIETSODA,EUDRINK,EUEAT,EUEXERCISE,EUEXFREQ,EUFASTFD,EUFASTFDFRQ,EUFFYDAY,EUFDSIT,EUFINLWGT,EUSNAP,EUGENHTH,EUGROSHP,EUHGT,EUINCLVL,EUINCOME2,EUMEAT,EUMILK,EUPRPMEL,EUSODA,EUSTORES,EUSTREASON,EUTHERM,EUWGT,EUWIC,EXINCOME1
0,20160101160045,1,2,26.6,3,3,5,40,0,0,0,-1,1,2,2,-1,2,-1,-1,1,2.529098e+07,2,3,1,66,7,1,1,2,1,2,2,4,2,165,-1,0
1,20160101160066,1,2,44.3,3,3,5,30,15,0,0,-1,2,1,1,2,2,-1,-1,1,6.301579e+06,2,4,1,69,7,1,1,2,1,-1,1,2,2,300,-1,0
2,20160101160069,1,2,24.5,3,5,-1,75,0,0,0,-1,2,2,1,1,2,-1,-1,1,9.545378e+06,1,2,1,64,7,2,1,2,1,-1,1,5,1,143,2,0
3,20160101160083,1,2,21.2,1,5,-1,165,0,0,2,-1,1,2,1,3,2,-1,-1,1,3.034797e+06,1,3,1,57,7,2,1,2,1,2,1,1,2,98,2,0
4,20160101160084,1,1,29.7,3,1,3,30,27,0,0,-1,1,1,2,-1,2,-1,-1,1,7.238641e+06,2,2,1,64,7,-1,2,2,3,2,1,1,-1,173,-1,0


In [402]:
raw_atussum.head()

,TUCASEID,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,TEMJOT,TRDPFTPT,TESCHENR,TESCHLVL,TRSPPRES,TESPEMPNOT,TRERNWA,TRCHILDNUM,TRSPFTPT,TEHRUSLT,TUDIARYDAY,TRHOLIDAY,TRTEC,TRTHH,t010101,t010102,t010201,t010299,t010301,t010399,t010401,t010499,t020101,t020102,t020103,t020104,t020199,t020201,t020202,t020203,...,t180704,t180799,t180801,t180802,t180803,t180804,t180805,t180806,t180807,t180899,t180901,t180902,t180903,t180904,t180905,t180999,t181001,t181002,t181099,t181101,t181201,t181202,t181203,t181204,t181205,t181301,t181302,t181399,t181401,t181499,t181501,t181599,t181601,t181801,t189999,t500101,t500103,t500105,t500106,t500107
0,20160101160045,2.458865e+07,-1,62,2,39,1,2,1,5,-1,-1,-1,-1,1,2,-1,0,-1,-1,6,0,-1,0,690,0,25,0,0,0,0,0,75,6,0,0,0,50,0,45,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20160101160066,5.445941e+06,-1,69,1,37,2,2,2,5,-1,-1,-1,-1,1,2,-1,0,-1,-1,1,0,30,0,600,0,20,0,0,0,0,0,60,0,0,0,0,150,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,20160101160069,8.782622e+06,0,24,2,39,2,2,1,5,-1,-1,2,-1,3,-1,-1,2,-1,-1,1,0,-1,380,940,0,120,0,0,0,0,0,0,0,0,30,0,75,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,60,0,0,0,0
3,20160101160083,3.035910e+06,8,31,2,40,1,2,2,1,2,2,2,-1,3,-1,46944,3,-1,32,1,0,-1,705,635,0,20,0,0,0,0,0,20,50,65,60,0,90,0,50,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,20160101160084,6.978586e+06,-1,59,2,39,1,2,1,1,1,2,-1,-1,1,2,30250,0,-1,12,1,0,-1,0,500,0,80,0,0,0,0,0,30,25,0,0,0,0,0,0,...,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0


In [403]:
raw_atusrost.head()

,TUCASEID,TULINENO,TEAGE,TERRP,TESEX,TXAGE,TXRRP,TXSEX
0,20160101160045,1,62,18,2,0,0,0
1,20160101160045,2,65,20,1,0,0,0
2,20160101160045,3,43,22,1,0,0,0
3,20160101160066,1,69,18,1,0,0,0
4,20160101160066,2,67,20,2,0,0,0


Check wie viele leere Einträge wir für ERBMI haben

In [404]:
merged_inner['ERBMI'].isna().sum()

795

### *Checken was für Datentypen wir in den Tabellen haben bzw. Gültig sind*

In [405]:
list(raw_atusrost.dtypes)

[dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64')]

In [406]:
list(raw_ehresp.dtypes)

[dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64')]

In [407]:
list(raw_atussum.dtypes)

[dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int6

### *Wie viele insgesamte Einträge wir in allen 3 Datenbanken?*

In [408]:
x=len(raw_atusrost)
y=len(raw_atussum)
z=len(raw_ehresp)
[x,y,z]

[27884, 10493, 10210]

### *Wie viele leere Einträge haben wir in allen 3 Datenbanken*

Für die Datenbank *Atussum*:

In [409]:
df_isna_atussum = raw_atussum.isna().sum()
df_isna_atussum = df_isna_atussum.value_counts().sum()
df_isna_atussum

423

Für die Datenbank *Ehresp*:

In [410]:
df_isna_ehresp= raw_ehresp.isna().sum()
df_isna_ehresp= df_isna_ehresp.value_counts().sum()
df_isna_ehresp

37

Für die Datenbank *Aturost*:

In [411]:
df_isna_aturost = raw_atusrost.isna().sum()
df_isna_aturost= df_isna_aturost.value_counts().sum()
df_isna_aturost

8

### Nach Duplikate in TUCASEID schauen

In [412]:
list(unique_everseen(duplicates(raw_atussum['TUCASEID'])))

[]

In [413]:
list(unique_everseen(duplicates(raw_ehresp['TUCASEID'])))

[]

In [414]:
list(duplicates(duplicates(raw_atusrost['TUCASEID'])))

[20160101160045,
 20160101160069,
 20160101160069,
 20160101160083,
 20160101160083,
 20160101160094,
 20160101160094,
 20160101160094,
 20160101160116,
 20160101160116,
 20160101160116,
 20160101160135,
 20160101160135,
 20160101160533,
 20160101160533,
 20160101160546,
 20160101160546,
 20160101160550,
 20160101160550,
 20160101160564,
 20160101160564,
 20160101160564,
 20160101160573,
 20160101160573,
 20160101160573,
 20160101160573,
 20160101160574,
 20160101160574,
 20160101160578,
 20160101160578,
 20160101160591,
 20160101160602,
 20160101160663,
 20160101160663,
 20160101160734,
 20160101160734,
 20160101160758,
 20160101160769,
 20160101160769,
 20160101160813,
 20160101160813,
 20160101160820,
 20160101160820,
 20160101160820,
 20160101160828,
 20160101160828,
 20160101160835,
 20160101160835,
 20160101160837,
 20160101160837,
 20160101160846,
 20160101160846,
 20160101160862,
 20160101160869,
 20160101160869,
 20160101160884,
 20160101160884,
 20160101160884,
 2016010116088

### Max und Minimum anzeigen zum überprüfen der Gültigen Werte

In [415]:
x = raw_ehresp['ERBMI'].min()
y = raw_ehresp['ERBMI'].max()
[x,y]

[-1.0, 61.8]

In [416]:
x = raw_atusrost.min()
y = raw_atusrost.max()
[x,y]

[TUCASEID    20160101160045
 TULINENO                 1
 TEAGE                    0
 TERRP                   18
 TESEX                    1
 TXAGE                    0
 TXRRP                    0
 TXSEX                    0
 dtype: int64, TUCASEID    20161212162509
 TULINENO                17
 TEAGE                   85
 TERRP                   40
 TESEX                    2
 TXAGE                   60
 TXRRP                    0
 TXSEX                   23
 dtype: int64]

In [417]:
x = raw_atusrost['TEAGE'].min()
y = raw_atusrost['TEAGE'].max()
[x,y]

[0, 85]

In [418]:
x = raw_atussum.min()
y = raw_atussum.max()
[x,y]

[TUCASEID      2.016010e+13
 TUFINLWGT     6.845048e+05
 TRYHHCHILD   -1.000000e+00
 TEAGE         1.500000e+01
 TESEX         1.000000e+00
                   ...     
 t500101       0.000000e+00
 t500103       0.000000e+00
 t500105       0.000000e+00
 t500106       0.000000e+00
 t500107       0.000000e+00
 Length: 423, dtype: float64, TUCASEID      2.016121e+13
 TUFINLWGT     1.307397e+08
 TRYHHCHILD    1.700000e+01
 TEAGE         8.500000e+01
 TESEX         2.000000e+00
                   ...     
 t500101       6.750000e+02
 t500103       3.000000e+02
 t500105       1.680000e+02
 t500106       1.790000e+02
 t500107       2.000000e+01
 Length: 423, dtype: float64]

### *Zusammenführen von den Datenbanken: Atussum & Ehresp in ein neues Dataframe merged_inner*

In [419]:
merged_inner = pd.merge(raw_atussum,raw_ehresp, on = 'TUCASEID', how ='outer')
merged_inner.head(30)

,TUCASEID,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,TEMJOT,TRDPFTPT,TESCHENR,TESCHLVL,TRSPPRES,TESPEMPNOT,TRERNWA,TRCHILDNUM,TRSPFTPT,TEHRUSLT,TUDIARYDAY,TRHOLIDAY,TRTEC,TRTHH,t010101,t010102,t010201,t010299,t010301,t010399,t010401,t010499,t020101,t020102,t020103,t020104,t020199,t020201,t020202,t020203,...,t500103,t500105,t500106,t500107,TULINENO,EEINCOME1,ERBMI,ERHHCH,ERINCOME,ERSPEMCH,ERTPREAT,ERTSEAT,ETHGT,ETWGT,EUDIETSODA,EUDRINK,EUEAT,EUEXERCISE,EUEXFREQ,EUFASTFD,EUFASTFDFRQ,EUFFYDAY,EUFDSIT,EUFINLWGT,EUSNAP,EUGENHTH,EUGROSHP,EUHGT,EUINCLVL,EUINCOME2,EUMEAT,EUMILK,EUPRPMEL,EUSODA,EUSTORES,EUSTREASON,EUTHERM,EUWGT,EUWIC,EXINCOME1
0,20160101160045,2.458865e+07,-1,62,2,39,1,2,1,5,-1,-1,-1,-1,1,2,-1,0,-1,-1,6,0,-1,0,690,0,25,0,0,0,0,0,75,6,0,0,0,50,0,45,...,0,0,0,0,1.0,2.0,26.6,3.0,3.0,5.0,40.0,0.0,0.0,0.0,-1.0,1.0,2.0,2.0,-1.0,2.0,-1.0,-1.0,1.0,2.529098e+07,2.0,3.0,1.0,66.0,7.0,1.0,1.0,2.0,1.0,2.0,2.0,4.0,2.0,165.0,-1.0,0.0
1,20160101160066,5.445941e+06,-1,69,1,37,2,2,2,5,-1,-1,-1,-1,1,2,-1,0,-1,-1,1,0,30,0,600,0,20,0,0,0,0,0,60,0,0,0,0,150,0,0,...,0,0,0,0,1.0,2.0,44.3,3.0,3.0,5.0,30.0,15.0,0.0,0.0,-1.0,2.0,1.0,1.0,2.0,2.0,-1.0,-1.0,1.0,6.301579e+06,2.0,4.0,1.0,69.0,7.0,1.0,1.0,2.0,1.0,-1.0,1.0,2.0,2.0,300.0,-1.0,0.0
2,20160101160069,8.782622e+06,0,24,2,39,2,2,1,5,-1,-1,2,-1,3,-1,-1,2,-1,-1,1,0,-1,380,940,0,120,0,0,0,0,0,0,0,0,30,0,75,0,0,...,0,0,0,0,1.0,2.0,24.5,3.0,5.0,-1.0,75.0,0.0,0.0,0.0,-1.0,2.0,2.0,1.0,1.0,2.0,-1.0,-1.0,1.0,9.545378e+06,1.0,2.0,1.0,64.0,7.0,2.0,1.0,2.0,1.0,-1.0,1.0,5.0,1.0,143.0,2.0,0.0
3,20160101160083,3.035910e+06,8,31,2,40,1,2,2,1,2,2,2,-1,3,-1,46944,3,-1,32,1,0,-1,705,635,0,20,0,0,0,0,0,20,50,65,60,0,90,0,50,...,0,0,0,0,1.0,2.0,21.2,1.0,5.0,-1.0,165.0,0.0,0.0,2.0,-1.0,1.0,2.0,1.0,3.0,2.0,-1.0,-1.0,1.0,3.034797e+06,1.0,3.0,1.0,57.0,7.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,98.0,2.0,0.0
4,20160101160084,6.978586e+06,-1,59,2,39,1,2,1,1,1,2,-1,-1,1,2,30250,0,-1,12,1,0,-1,0,500,0,80,0,0,0,0,0,30,25,0,0,0,0,0,0,...,0,0,0,0,1.0,1.0,29.7,3.0,1.0,3.0,30.0,27.0,0.0,0.0,-1.0,1.0,1.0,2.0,-1.0,2.0,-1.0,-1.0,1.0,7.238641e+06,2.0,2.0,1.0,64.0,7.0,-1.0,2.0,2.0,3.0,2.0,1.0,1.0,-1.0,173.0,-1.0,0.0
5,20160101160094,5.191610e+06,4,16,2,36,3,1,1,5,-1,-1,1,1,3,-1,-1,4,-1,-1,1,0,-1,0,565,0,55,0,0,0,0,0,0,0,0,0,0,90,10,0,...,0,0,0,0,1.0,2.0,30.9,3.0,5.0,-1.0,120.0,9.0,0.0,0.0,2.0,1.0,1.0,1.0,6.0,1.0,1.0,2.0,1.0,5.298292e+06,1.0,2.0,2.0,60.0,7.0,2.0,-1.0,-1.0,2.0,1.0,-1.0,-1.0,-1.0,158.0,1.0,0.0
6,20160101160116,9.391848e+06,5,43,2,43,1,2,1,5,-1,-1,2,-1,1,1,-1,3,1,-1,3,0,-1,120,435,0,10,0,0,0,0,0,0,0,0,0,0,80,0,0,...,0,0,0,0,1.0,1.0,18.3,3.0,1.0,5.0,45.0,0.0,0.0,0.0,-1.0,2.0,2.0,1.0,3.0,2.0,-1.0,-1.0,1.0,9.648727e+06,2.0,1.0,1.0,65.0,7.0,-1.0,1.0,2.0,1.0,-1.0,1.0,2.0,2.0,110.0,2.0,0.0
7,20160101160135,2.023918e+06,0,34,2,39,1,2,1,5,-1,-1,2,-1,3,-1,-1,3,-1,-1,1,0,-1,615,645,0,20,0,0,0,0,0,180,90,0,0,0,5,0,30,...,0,0,0,0,1.0,2.0,40.8,3.0,5.0,-1.0,40.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,7.0,1.0,1.0,2.0,1.0,2.018001e+06,1.0,3.0,1.0,65.0,7.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,245.0,1.0,0.0
8,20160101160139,9.112257e+06,-1,63,1,46,1,2,1,5,-1,-1,-1,-1,1,1,-1,0,1,-1,1,0,-1,0,510,0,0,0,0,0,0,0,0,0,0,0,0,60,0,0,...,0,0,0,0,1.0,1.0,27.6,3.0,1.0,5.0,60.0,0.0,0.0,0.0,-1.0,2.0,2.0,1.0,7.0,1.0,1.0,2.0,1.0,8.881153e+06,2.0,2.0,2.0,74.0,7.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,1.0,215.0,-1.0,0.0
9,20160101160533,2.742447e+06,7,39,2,40,1,2,1,1,2,1,2,-1,1,1,-1,2,1,46,1,0,-1,520,670,0,0,0,0,0,90,0,120,0,0,0,0,40,0,0,...,0,0,0,0,1.0,2.0,28.3,3.0,3.0,5.0,120.0,30.0,0.0,0.0,-1.0,1.0,1.0,1.0,3.0,1.0,5.0,2.0,1.0,2.826905e+06,2.0,3.0,2.0,65.0,7.0,1.0,-1.0,-1.0,2.0,2.0,-1.0,-1.0,-1.0,170.0,2.0,0.0


### *Alle Ungültige Werte (0,-1) mit Nana ersezten*

Funktion zur Löschung der unglütigen Werte < 0

In [420]:
def unglueltige_Werte_loeschen(df,c):
    for x in df[c]:
        if x < 1: 
            df[c] = df[c].replace([x],np.NaN)

Funktionsparameter übergeben

In [421]:
unglueltige_Werte_loeschen(merged_inner, 'ERBMI')

list(merged_inner['ERBMI'][::])

[26.6,
 44.3,
 24.5,
 21.2,
 29.7,
 30.9,
 18.3,
 40.8,
 27.6,
 28.3,
 28.2,
 41.8,
 26.6,
 19.9,
 28.3,
 27.4,
 23.7,
 39.4,
 36.6,
 24.7,
 38.0,
 40.3,
 45.9,
 46.3,
 20.5,
 40.3,
 31.3,
 28.2,
 24.6,
 38.4,
 24.2,
 30.3,
 29.0,
 31.6,
 27.5,
 27.6,
 nan,
 34.7,
 28.3,
 37.1,
 23.6,
 22.1,
 18.2,
 nan,
 30.8,
 27.2,
 17.7,
 32.1,
 33.0,
 22.6,
 nan,
 nan,
 23.5,
 33.9,
 18.8,
 20.9,
 23.7,
 27.4,
 21.1,
 22.8,
 42.1,
 21.8,
 nan,
 29.3,
 22.1,
 31.3,
 24.4,
 35.9,
 28.5,
 29.3,
 19.5,
 21.1,
 24.4,
 35.9,
 37.1,
 27.1,
 nan,
 nan,
 21.0,
 26.3,
 19.7,
 30.5,
 nan,
 35.4,
 nan,
 35.2,
 33.9,
 27.9,
 23.6,
 19.1,
 27.8,
 25.6,
 20.0,
 18.0,
 27.2,
 22.6,
 28.6,
 nan,
 23.7,
 23.7,
 25.0,
 32.5,
 25.0,
 31.0,
 42.0,
 28.2,
 25.8,
 18.8,
 27.4,
 41.6,
 34.9,
 30.1,
 28.0,
 32.1,
 nan,
 28.3,
 27.0,
 nan,
 37.3,
 24.1,
 21.2,
 45.7,
 27.5,
 27.4,
 45.2,
 23.6,
 21.2,
 41.3,
 19.8,
 28.0,
 35.9,
 28.3,
 34.7,
 28.3,
 35.5,
 26.6,
 35.2,
 31.9,
 37.8,
 41.7,
 23.5,
 22.6,
 23.2,
 28.3,
 34.

### *Spliten zwischen Mann und Frau in zwei neue Datenbanken*

In [422]:
merged_inner_male = merged_inner[merged_inner['TESEX'] == 1 ]
merged_inner_female = merged_inner[merged_inner['TESEX'] == 2]

Print Tabellen


In [423]:
[merged_inner_male, merged_inner_female]

[             TUCASEID     TUFINLWGT  TRYHHCHILD  ...  EUWGT  EUWIC  EXINCOME1
 1      20160101160066  5.445941e+06          -1  ...  300.0   -1.0        0.0
 8      20160101160139  9.112257e+06          -1  ...  215.0   -1.0        0.0
 10     20160101160546  3.165768e+06           1  ...  208.0    2.0        0.0
 16     20160101160573  9.150706e+06          16  ...  170.0    2.0       73.0
 17     20160101160574  1.299119e+06          13  ...  267.0    2.0        0.0
 ...               ...           ...         ...  ...    ...    ...        ...
 10482  20161212162015  2.775790e+06          -1  ...  158.0   -1.0        0.0
 10483  20161212162045  1.985660e+07          17  ...  168.0   -1.0       73.0
 10486  20161212162212  1.986985e+07          -1  ...  180.0   -1.0        0.0
 10489  20161212162357  4.899612e+06          -1  ...  174.0   -1.0        0.0
 10490  20161212162426  1.597792e+07          -1  ...  158.0   -1.0        0.0
 
 [4670 rows x 459 columns],
              TUCASEID

### *Wie viele Werte haben wir für Man oder Frau in den Datenbanken?*

In [424]:
df_male = raw_atussum[raw_atussum['TESEX'] == 1 ].value_counts().sum() 
df_female = raw_atussum[raw_atussum['TESEX'] == 2].value_counts().sum()
[df_female, df_male]

[5823, 4670]

Validieren ob ein Fehlerhafter Wert noch exisitiert

In [425]:
TEAGE = raw_atussum['TEAGE'] < 0
print(TEAGE)

0        False
1        False
2        False
3        False
4        False
         ...  
10488    False
10489    False
10490    False
10491    False
10492    False
Name: TEAGE, Length: 10493, dtype: bool


### Frauen und Männern leere Einträge für ERBMI

In [426]:
merged_inner_male.describe(include='all').T
merged_inner_male['ERBMI'].isna().sum()
merged_inner_male.isna().sum()

TUCASEID        0
TUFINLWGT       0
TRYHHCHILD      0
TEAGE           0
TESEX           0
             ... 
EUSTREASON    127
EUTHERM       127
EUWGT         127
EUWIC         127
EXINCOME1     127
Length: 459, dtype: int64

In [427]:
merged_inner_female.describe(include='all').T
merged_inner_female['ERBMI'].isna().sum()
merged_inner_female.isna().sum()

TUCASEID        0
TUFINLWGT       0
TRYHHCHILD      0
TEAGE           0
TESEX           0
             ... 
EUSTREASON    156
EUTHERM       156
EUWGT         156
EUWIC         156
EXINCOME1     156
Length: 459, dtype: int64

## **Visualisieren Sie die Ergebnisse**

In [428]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=merged_inner[merged_inner['TESEX'] == 1] ['ERBMI'] , name='Male'))
fig.add_trace(go.Histogram(x=merged_inner[merged_inner['TESEX'] == 2] ['ERBMI'] , name='Female'))
fig.update_traces(opacity = 0.75, xbins=dict(start=0, end= 200, size=5 ))
fig.show()


In [429]:
fig= go.Figure()
fig.add_trace(go.Histogram(x=merged_inner[merged_inner['TESEX'] == 1] ['TEAGE'] , name='Male'))
fig.add_trace(go.Histogram(x=merged_inner[merged_inner['TESEX'] == 2] ['TEAGE'] , name='Female'))
fig.update_traces(opacity = 0.75, xbins=dict(start=15, end= 65, size=5 ))
fig.show()


In [430]:
fig = px.scatter(merged_inner, x="ERBMI", y="TEAGE",
                 title="Default Display with Gaps")
fig.show()

## **Bereinigen der Daten, indem ungültige Einträge entfernen oder mit geeigneten Werten füllen**

In [431]:
def calc_data_stats(df, feature_name):
    return pd.DataFrame(dict(
        feature=feature_name,
        mean=df.mean(),
        median=df.median(),
        #mode=df.mode(numeric_only=True).T[0],
        mode=df.mode(),
        std=df.std(),
    ))
ERBMI_stats = calc_data_stats(merged_inner['ERBMI'],'ERBMI STATS' )

In [432]:
df_meanF = merged_inner_female.copy()
mean_val_W = df_meanF['ERBMI'].mean()
df_meanF['ERBMI'].fillna(df_meanF['ERBMI'].mean(), inplace=True)
#list(df_meanF['ERBMI'])

In [433]:
print(df_meanF['ERBMI'].mean())

27.632843511450382


In [434]:
df_meanM = merged_inner_male.copy()
mean_val_M=df_meanM['ERBMI'].mean()
df_meanM['ERBMI'].fillna(df_meanM['ERBMI'].mean(), inplace=True)
#list(df_meanM['ERBMI'])

In [435]:
print(df_meanM['ERBMI'].mean())

27.977613279497564


In [436]:
df_mean = merged_inner.copy()
df_mean
print(df_mean['ERBMI'].mean())
df_mean['ERBMI'].fillna(df_mean['ERBMI'].mean(), inplace=True)

27.791328108888315


In [437]:
def Fill_Mean(df,ERBMI_val,c):
    for x in df[c]:
        if x < 1: 
            df[c] = df[c].replace([x],ERBMI_val)

In [438]:
fig= go.Figure()
fig.add_trace(go.Histogram(x=merged_inner[merged_inner['TESEX'] == 1] ['ERBMI'] , name='Male'))
fig.add_trace(go.Histogram(x=merged_inner[merged_inner['TESEX'] == 2] ['ERBMI'] , name='Female'))
fig.update_traces(opacity = 0.75, xbins=dict(start=-5, end= 65, size=5 ))
fig.show()


In [439]:
fig= go.Figure()
fig.add_trace(go.Histogram(x=df_meanM[df_meanM['TESEX'] == 1] ['ERBMI'] , name='Male'))
fig.add_trace(go.Histogram(x=df_meanF[df_meanF['TESEX'] == 2] ['ERBMI'] , name='Female'))
fig.update_traces(opacity = 0.75, xbins=dict(start=-5, end= 65, size=5 ))
fig.show()

In [440]:
fig= go.Figure()
fig.add_trace(go.Histogram(x=df_mean[df_mean['TESEX'] == 1] ['ERBMI'] , name='Male'))
fig.add_trace(go.Histogram(x=df_mean[df_mean['TESEX'] == 2] ['ERBMI'] , name='Female'))
fig.update_traces(opacity = 0.75, xbins=dict(start=-5, end= 65, size=5 ))
fig.show()


## **Alle Prädiktionen die mit dem BMI oder Gewicht verbunden sind, wurden gelöscht.**


In [441]:
merged_male = merged_inner_male.drop(columns=['ETWGT', 'EUWGT'] )
merged_female = merged_inner_female.drop(columns=['ETWGT', 'EUWGT'])

# **Prädiktion**

## Vergleichen

In [442]:
#X = df_meanM[['EUEXFREQ',  'EUHGT', 'ERTPREAT']]
#y = df_meanM['ERBMI']

# fit a glm model
#reg = LinearRegression()
#reg.fit(X,y)

In [443]:
#y_pred = reg.predict(X)
#residual = y - y_pred
#plt.scatter(y_pred,residual)

In [444]:
#list(df_meanF['EUHGT'][::])

#df_meanM[]df_meanM[]'EUHGT']<0

## Ansatz